In [ ]:
#default_exp transforms

# Audio Transforms
> Data transformations made specifically for audio.

In [ ]:
#export
from fastai2.basics import *
import librosa
import torchaudio

In [ ]:
from IPython.display import Audio
import matplotlib.pyplot as plt
import seaborn as sns

# Normalize Audio

In [ ]:
#export
class NormalizeAudio(Transform):
    def encodes(self, sig:ndarray):
        return librosa.util.normalize(sig)

# MuLawEncoding

In [ ]:
#export
class MuLawEncoding(Transform):
    order = 11
    def __init__(self, quantization_channels:int=256):
        self.MuLawEnc = torchaudio.transforms.MuLawEncoding(quantization_channels)
        self.MuLawDec = torchaudio.transforms.MuLawDecoding(quantization_channels)
    def encodes(self, sig:Tensor):
        print("enc")
        return self.MuLawEnc(sig)
    def decodes(self, sig:Tensor):
        print("dec")
        return self.MuLawDec(sig)